# **Limpeza e Pré-processamento de dados do _IFCE em Números_**

## **Equipe:**
- ### Antonia Raiane Santos Araujo Cruz - 507569
- ### Cristiano de Carvalho Martins Filho - 507542
- ### Ronney Nery Silva - 486342

## **Objetivo**
- ### Proporcionar Análises e Visualizações sobre os dados acadêmicos do _IFCE em Números_

## **Contextualização**

### O projeto é baseado em informações acadêmicas contidas no site _[IFCE em Números](https://ifceemnumeros.ifce.edu.br/)_.
- ### Pretende-se mostrar visualizações sobre os dados acadêmicos do Instituto Federal de Educação (IFCE).
- ### Apresentaremos a  distribuição do índice de evasão por campus, sexo e curso, além de verificar se a evasão é maior no público de alunos veteranos ou ingressantes.
- ### Serão apresentados também visualizações sobre a evasão considerando a cidade de moradia do aluno, logo que  muitos alunos se deslocam da sua cidade para um outro município, que possui um campus do IFCE. 

## **Questões**

1. ### Quais campus possuem mais alunos? 
2. ### Existe relação entre o aluno ser de uma localidade diferente da cidade onde o campus está localizado com a evasão?
3. ### Quais tipos de cursos são mais ofertados de acordo com a modalidade de ensino?
4. ### Existe relação com a evasão e o gênero com as modalidades de curso? 
5. ### Existe uma tendência de evasão do sexo feminino nos cursos de tecnologia? 
6. ### Os alunos ingressantes evadem mais que os alunos veteranos? 
7. ### É possível observar em que período do curso os alunos costumam evadir-se mais?

## **Link Dataset**
- ### [Download dataset original](https://drive.google.com/file/d/1jGUSPqozLJH9I-XdljMY84gW4-5qvDbc/view?usp=sharing)
- ### [Download dataset tratado](https://drive.google.com/file/d/1HituLXZZ8QKqftIoe1Duotytfcjtyvby/view?usp=sharing)

---
## **1. Importação das bibliotecas**

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

# Define o número mínimo de linhas na visualização do DataFrame
pd.set_option("min_rows", 6)
pd.set_option("max_rows", 15)

---
## **2. Leitura do dataset**

In [ ]:
# Lendo o dataset do arquivo obtido
df_dados = pd.read_csv('ifce_2015_2019_superior_tratado.csv', encoding = 'UTF-8')
# Analisando a estrutura do dataset
df_dados.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426438 entries, 0 to 426437
Data columns (total 35 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   Unnamed: 0                        426438 non-null  int64 
 1   campus                            426438 non-null  object
 2   cidade_aluno                      426438 non-null  object
 3   curso                             426438 non-null  object
 4   df_final_periodo                  426438 non-null  object
 5   df_inicio_periodo                 426438 non-null  object
 6   estado_aluno                      426438 non-null  object
 7   etnia                             426438 non-null  object
 8   faixa_etaria                      426438 non-null  object
 9   ingressante                       426438 non-null  int64 
 10  matricula                         426438 non-null  object
 11  modalidade_ensino                 426438 non-null  object
 12  mo

---
## **3. Operações de transformação dos dados**
## **3.1 Operações em colunas**

In [ ]:
# Eliminando colunas não utilizadas
df_dados.drop(columns=['df_final_periodo', 'faixa_etaria', 'ingressante', 'previsão_termino', \
                       'situacao_matricula', 'situacao_matricula_grupo', 'situacao_matricula_periodo', \
                       'situacao_matricula_periodo_grupo', 'tipo_situacao_matricula', 'ingressante_rotulo', \
                       'matricula_retida_periodo_letivo', 'ultima_alteracao_situacao', 'idade'], \
              axis=0, inplace=True)
# Renomeia algumas colunas
df_dados.rename(columns={'Unnamed: 0': 'id', 'periodo_letivo_num': 'periodo_letivo', \
                         'qtde_matriculas_evadidas': 'qtde_evadidos'}, \
                inplace=True)
# Mudando o tipo da coluna 'periodo_letivo'
df_dados['periodo_letivo'] = df_dados['periodo_letivo'].astype(str)
# Mudando o tipo da coluna 'matricula'
df_dados['matricula'] = df_dados['matricula'].astype(str)
# Altera a coluna 'cidade_aluno' para letras maiúsculas
df_dados['cidade_aluno'] = df_dados['cidade_aluno'].str.upper()
# Muda os valores curtos de 'sexo' para uma descrição completa
df_dados['sexo'].replace({'F': 'Feminino', 'M': 'Masculino'}, inplace=True)
df_dados

,id,campus,cidade_aluno,curso,df_inicio_periodo,estado_aluno,etnia,matricula,modalidade_ensino,modalidade_curso,cidade_campus,nivel_ensino,periodo_letivo,sexo,qtde_ingressantes,qtde_matriculados,qtde_matriculados_cursando,qtde_matriculados_emconclusao,qtde_matriculados_formados,qtde_matriculados_suspensos,qtde_evadidos,cidades_iguais
0,74,CAMPUS MARACANAÚ,LIMOEIRO DO NORTE,ESPECIALIZAÇÃO EM EDUCAÇÃO AMBIENTAL,31/07/2010,CE,Branca,201028111738,A Distância,Especialização,MARACANAÚ,Pós-graduação,20102,Feminino,1,1,0,0,1,0,0,0
1,75,CAMPUS MARACANAÚ,SÃO GONÇALO DO AMARANTE,ESPECIALIZAÇÃO EM EDUCAÇÃO AMBIENTAL,31/07/2010,CE,Branca,201028111312,A Distância,Especialização,MARACANAÚ,Pós-graduação,20102,Feminino,1,1,0,0,1,0,0,0
2,442,CAMPUS IGUATU,QUIXELÔ,TÉCNICO EM AGROINDÚSTRIA,09/03/2015,CE,Parda,20151092290094,Presencial,Concomitante,IGUATU,Técnico,20151,Masculino,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426435,669469,CAMPUS IGUATU,IGUATU,LICENCIATURA EM GEOGRAFIA,02/08/2021,CE,Branca,20192094000287,Presencial,Licenciatura,IGUATU,Graduação,20211,Feminino,0,1,1,0,0,0,0,1
426436,669470,CAMPUS IGUATU,CEDRO,LICENCIATURA EM GEOGRAFIA,02/08/2021,CE,Branca,20202094000393,Presencial,Licenciatura,IGUATU,Graduação,20211,Feminino,0,1,1,0,0,0,0,0
426437,669471,CAMPUS IGUATU,IGUATU,LICENCIATURA EM GEOGRAFIA,02/08/2021,CE,Branca,20202094000180,Presencial,Licenciatura,IGUATU,Graduação,20211,Feminino,0,1,1,0,0,0,0,1


### **3.2 Eliminação de registros com repetições**

In [ ]:
query = df_dados.groupby(by=['campus', 'cidade_aluno', 'curso', 'estado_aluno', 'etnia', 'matricula', \
                             'modalidade_ensino', 'modalidade_curso', 'cidade_campus', 'nivel_ensino', \
                             'periodo_letivo', 'sexo', 'qtde_ingressantes', 'qtde_matriculados', \
                             'qtde_matriculados_cursando', 'qtde_matriculados_emconclusao', \
                             'qtde_matriculados_formados', 'qtde_matriculados_suspensos', 'qtde_evadidos'], \
                         as_index=False).agg({'id': pd.Series.nunique})
query.rename(columns={'id': 'qtde_repeticoes'}, inplace=True)
query['qtde_repeticoes'].value_counts()

1    420087
2      2501
3       205
4        67
6        56
5        26
Name: qtde_repeticoes, dtype: int64

In [ ]:
df_dados.drop_duplicates(subset=['campus', 'cidade_aluno', 'curso', 'estado_aluno', 'etnia', 'matricula', \
                                 'modalidade_ensino', 'modalidade_curso', 'cidade_campus', 'nivel_ensino', \
                                 'periodo_letivo', 'sexo', 'qtde_ingressantes', 'qtde_matriculados', \
                                 'qtde_matriculados_cursando', 'qtde_matriculados_emconclusao', \
                                 'qtde_matriculados_formados', 'qtde_matriculados_suspensos', 'qtde_evadidos'], \
                         keep='first', inplace=True)
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422942 entries, 0 to 426437
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   id                             422942 non-null  int64 
 1   campus                         422942 non-null  object
 2   cidade_aluno                   422942 non-null  object
 3   curso                          422942 non-null  object
 4   df_inicio_periodo              422942 non-null  object
 5   estado_aluno                   422942 non-null  object
 6   etnia                          422942 non-null  object
 7   matricula                      422942 non-null  object
 8   modalidade_ensino              422942 non-null  object
 9   modalidade_curso               422942 non-null  object
 10  cidade_campus                  422942 non-null  object
 11  nivel_ensino                   422942 non-null  object
 12  periodo_letivo                 422942 non-nu

---
## **4. Criação do banco de dados**
#### Fonte: http://pythonclub.com.br/gerenciando-banco-dados-sqlite3-python-parte1.html

In [ ]:
conexao_bd = sqlite3.connect('ifce_evasao.db')

---
## **5. Criação de tabelas de categorias**
#### Realiza consultas para construir tabelas com valores únicos presentes nos vários campos categóricos do dataset fonte.
#### O objetivo destas tabelas é facilitar a utilização de controles para filtragem dos dados dos gráficos por meio de controles.
#### Fonte: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

### **5.1 Campus**

In [ ]:
# Obtém uma lista com valores distintos de 'campus'
df_campus = df_dados.groupby(by=['campus'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_campus.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'campus': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_campus = pd.concat([linha_vazia, df_campus], ignore_index=True)
# Armazena o dataframe no banco de dados
df_campus.to_sql('tbl_campus', conexao_bd, index_label='id')
df_campus.head()

,campus
0,
1,CAMPUS ACARAÚ
2,CAMPUS ACOPIARA
3,CAMPUS ARACATI
4,CAMPUS BATURITÉ


### **5.2 Cidade do Aluno**

In [ ]:
# Obtém uma lista com valores distintos de 'cidade_aluno'
df_cidade_aluno = df_dados.groupby(by=['cidade_aluno'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_cidade_aluno.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'cidade_aluno': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_cidade_aluno = pd.concat([linha_vazia, df_cidade_aluno], ignore_index=True)
# Armazena o dataframe no banco de dados
df_cidade_aluno.to_sql('tbl_cidade_aluno', conexao_bd, index_label='id')
df_cidade_aluno.head()

,cidade_aluno
0,
1,ABAIARA
2,ACARAPE
3,ACARAÚ
4,ACOPIARA


### **5.3 Cidade do Campus**

In [ ]:
# Obtém uma lista com valores distintos de 'cidade_campus'
df_cidade_campus = df_dados.groupby(by=['cidade_campus'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_cidade_campus.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'cidade_campus': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_cidade_campus = pd.concat([linha_vazia, df_cidade_campus], ignore_index=True)
# Armazena o dataframe no banco de dados
df_cidade_campus.to_sql('tbl_cidade_campus', conexao_bd, index_label='id')
df_cidade_campus.head()

,cidade_campus
0,
1,ACARAÚ
2,ACOPIARA
3,ARACATI
4,BATURITÉ


### **5.4 Curso**

In [ ]:
# Obtém uma lista com valores distintos de 'curso'
df_curso = df_dados.groupby(by=['curso'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_curso.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'curso': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_curso = pd.concat([linha_vazia, df_curso], ignore_index=True)
# Armazena o dataframe no banco de dados
df_curso.to_sql('tbl_curso', conexao_bd, index_label='id')
df_curso.head()

,curso
0,
1,APERFEIÇOAMENTO EM DOCÊNCIA NA EDUCAÇÃO PROFIS...
2,BACHARELADO EM AGRONOMIA
3,BACHARELADO EM CIÊNCIA DA COMPUTAÇÃO
4,BACHARELADO EM ENGENHARIA AGRÍCOLA


### **5.5 Estado do Aluno**

In [ ]:
# Obtém uma lista com valores distintos de 'estado_aluno'
df_estado_aluno = df_dados.groupby(by=['estado_aluno'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_estado_aluno.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'estado_aluno': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_estado_aluno = pd.concat([linha_vazia, df_estado_aluno], ignore_index=True)
# Armazena o dataframe no banco de dados
df_estado_aluno.to_sql('tbl_estado_aluno', conexao_bd, index_label='id')
df_estado_aluno.head()

,estado_aluno
0,
1,AL
2,AM
3,BA
4,CE


### **5.6 Etnia**

In [ ]:
# Obtém uma lista com valores distintos de 'etnia'
df_etnia = df_dados.groupby(by=['etnia'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_etnia.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'etnia': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_etnia = pd.concat([linha_vazia, df_etnia], ignore_index=True)
# Armazena o dataframe no banco de dados
df_etnia.to_sql('tbl_etnia', conexao_bd, index_label='id')
df_etnia.head()

,etnia
0,
1,Amarela
2,Branca
3,Indígena
4,Parda


### **5.7 Modalidade do Curso**

In [ ]:
# Obtém uma lista com valores distintos de 'modalidade_curso'
df_modalidade_curso = df_dados.groupby(by=['modalidade_curso'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_modalidade_curso.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'modalidade_curso': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_modalidade_curso = pd.concat([linha_vazia, df_modalidade_curso], ignore_index=True)
# Armazena o dataframe no banco de dados
df_modalidade_curso.to_sql('tbl_modalidade_curso', conexao_bd, index_label='id')
df_modalidade_curso.head()

,modalidade_curso
0,
1,Bacharelado
2,Concomitante
3,Especialização
4,Integrado


### **5.8 Modalidade de Ensino**

In [ ]:
# Obtém uma lista com valores distintos de 'modalidade_ensino'
df_modalidade_ensino = df_dados.groupby(by=['modalidade_ensino'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_modalidade_ensino.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'modalidade_ensino': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_modalidade_ensino = pd.concat([linha_vazia, df_modalidade_ensino], ignore_index=True)
# Armazena o dataframe no banco de dados
df_modalidade_ensino.to_sql('tbl_modalidade_ensino', conexao_bd, index_label='id')
df_modalidade_ensino.head()

,modalidade_ensino
0,
1,A Distância
2,Presencial


### **5.9 Nível de Ensino**

In [ ]:
# Obtém uma lista com valores distintos de 'nivel_ensino'
df_nivel_ensino = df_dados.groupby(by=['nivel_ensino'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_nivel_ensino.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'nivel_ensino': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_nivel_ensino = pd.concat([linha_vazia, df_nivel_ensino], ignore_index=True)
# Armazena o dataframe no banco de dados
df_nivel_ensino.to_sql('tbl_nivel_ensino', conexao_bd, index_label='id')
df_nivel_ensino.head()

,nivel_ensino
0,
1,Graduação
2,Pós-graduação
3,Técnico


### **5.10 Período Letivo**

In [ ]:
# Obtém uma lista com valores distintos de 'periodo_letivo'
df_periodo_letivo = df_dados.groupby(by=['periodo_letivo'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_periodo_letivo.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'periodo_letivo': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_periodo_letivo = pd.concat([linha_vazia, df_periodo_letivo], ignore_index=True)
# Armazena o dataframe no banco de dados
df_periodo_letivo.to_sql('tbl_periodo_letivo', conexao_bd, index_label='id')
df_periodo_letivo.head()

,periodo_letivo
0,
1,20091
2,20092
3,20101
4,20102


### **5.11 Sexo**

In [ ]:
# Obtém uma lista com valores distintos de 'sexo'
df_sexo = df_dados.groupby(by=['sexo'], as_index=False).agg({'id': pd.Series.count})
# Elimina a coluna "index"
df_sexo.drop(columns=['id'], inplace=True)
# Cria um registro com texto vazio para retirar filtro
linha_vazia = pd.DataFrame.from_dict({'sexo': ['']})
# Adiciona a "linha vazia" ao dataframe dos valores
df_sexo = pd.concat([linha_vazia, df_sexo], ignore_index=True)
# Armazena o dataframe no banco de dados
df_sexo.to_sql('tbl_sexo', conexao_bd, index_label='id')
df_sexo.head()

,sexo
0,
1,Feminino
2,Masculino


---
## **6. Consultas**
### **6.1 Quantidade de Alunos por Curso e Campus**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['curso', 'campus'], as_index=False).agg({'matricula': 'nunique'})
# Renomeia a coluna da contagem
df_consulta.rename(columns={'matricula': 'qtde_alunos'}, inplace=True)
# Armazena o dataframe no banco de dados
df_consulta.to_sql('qry_qtde_alunos_curso_campus', conexao_bd, index_label='id')
df_consulta

,curso,campus,qtde_alunos
0,APERFEIÇOAMENTO EM DOCÊNCIA NA EDUCAÇÃO PROFIS...,CAMPUS FORTALEZA,56
1,BACHARELADO EM AGRONOMIA,CAMPUS LIMOEIRO DO NORTE,233
2,BACHARELADO EM CIÊNCIA DA COMPUTAÇÃO,CAMPUS ARACATI,519
...,...,...,...
304,TÉCNICO EM TELECOMUNICAÇÕES,CAMPUS CANINDÉ,171
305,TÉCNICO EM TELECOMUNICAÇÕES,CAMPUS FORTALEZA,572
306,TÉCNICO EM ZOOTECNIA,CAMPUS IGUATU,239


### **6.2 Quantidade de Alunos por Etnia e Campus**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['etnia', 'campus'], as_index=False).agg({'matricula': 'nunique'})
# Renomeia a coluna da contagem
df_consulta.rename(columns={'matricula': 'qtde_alunos'}, inplace=True)
# Armazena o dataframe no banco de dados
df_consulta.to_sql('qry_qtde_alunos_etnia_campus', conexao_bd, index_label='id')
df_consulta

,etnia,campus,qtde_alunos
0,Amarela,CAMPUS ACARAÚ,25
1,Amarela,CAMPUS ACOPIARA,5
2,Amarela,CAMPUS ARACATI,30
...,...,...,...
155,Preta,CAMPUS TIANGUÁ,116
156,Preta,CAMPUS UBAJARA,80
157,Preta,CAMPUS UMIRIM,42


### **6.3 Quantidade de Alunos por Modalidade de Ensino e Campus**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['modalidade_ensino', 'campus'], as_index=False).agg({'matricula': 'nunique'})
# Renomeia a coluna da contagem
df_consulta.rename(columns={'matricula': 'qtde_alunos'}, inplace=True)
# Armazena o dataframe no banco de dados
df_consulta.to_sql('qry_qtde_alunos_modalidade_ensino_campus', conexao_bd, index_label='id')
df_consulta

,modalidade_ensino,campus,qtde_alunos
0,A Distância,CAMPUS CANINDÉ,33
1,A Distância,CAMPUS CRATO,23
2,A Distância,CAMPUS FORTALEZA,1575
...,...,...,...
36,Presencial,CAMPUS TIANGUÁ,2265
37,Presencial,CAMPUS UBAJARA,1646
38,Presencial,CAMPUS UMIRIM,824


### **6.4 Quantidade de Alunos por Nível de Ensino e Campus**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['nivel_ensino', 'campus'], as_index=False).agg({'matricula': 'nunique'})
# Renomeia a coluna da contagem
df_consulta.rename(columns={'matricula': 'qtde_alunos'}, inplace=True)
# Armazena o dataframe no banco de dados
df_consulta.to_sql('qry_qtde_alunos_nivel_ensino_campus', conexao_bd, index_label='id')
df_consulta

,nivel_ensino,campus,qtde_alunos
0,Graduação,CAMPUS ACARAÚ,1129
1,Graduação,CAMPUS ACOPIARA,161
2,Graduação,CAMPUS ARACATI,1372
...,...,...,...
76,Técnico,CAMPUS TIANGUÁ,1063
77,Técnico,CAMPUS UBAJARA,548
78,Técnico,CAMPUS UMIRIM,588


### **6.5 Quantidade de Alunos por Sexo e Campus**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['sexo', 'campus'], as_index=False).agg({'matricula': 'nunique'})
# Renomeia a coluna da contagem
df_consulta.rename(columns={'matricula': 'qtde_alunos'}, inplace=True)
# Armazena o dataframe no banco de dados
df_consulta.to_sql('qry_qtde_alunos_sexo_campus', conexao_bd, index_label='id')
df_consulta

,sexo,campus,qtde_alunos
0,Feminino,CAMPUS ACARAÚ,1506
1,Feminino,CAMPUS ACOPIARA,260
2,Feminino,CAMPUS ARACATI,1548
...,...,...,...
61,Masculino,CAMPUS TIANGUÁ,1149
62,Masculino,CAMPUS UBAJARA,529
63,Masculino,CAMPUS UMIRIM,419


### **6.6 Quantidade de Alunos por Período, Sexo e Campus**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['periodo_letivo', 'nivel_ensino', 'campus', 'sexo', 'etnia'], \
                               as_index=False).agg({'matricula': 'nunique'})
# Renomeia a coluna da contagem
df_consulta.rename(columns={'matricula': 'qtde_alunos'}, inplace=True)
# Armazena o dataframe no banco de dados
df_consulta.to_sql('qry_qtde_alunos_periodo', conexao_bd, index_label='id')
df_consulta

,periodo_letivo,nivel_ensino,campus,sexo,etnia,qtde_alunos
0,20091,Graduação,CAMPUS CEDRO,Feminino,Amarela,1
1,20091,Graduação,CAMPUS CEDRO,Feminino,Branca,4
2,20091,Graduação,CAMPUS CEDRO,Feminino,Parda,7
...,...,...,...,...,...,...
9373,20212,Técnico,CAMPUS UMIRIM,Masculino,Indígena,1
9374,20212,Técnico,CAMPUS UMIRIM,Masculino,Parda,48
9375,20212,Técnico,CAMPUS UMIRIM,Masculino,Preta,1


### **6.7 Quantidade de Alunos Evadidos**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['periodo_letivo', 'nivel_ensino', 'modalidade_curso', 'curso', 'campus', \
                                   'sexo', 'etnia'], \
                               as_index=False).agg({'qtde_evadidos': 'sum'})
# Retira os resultados zerados
df_consulta = df_consulta[df_consulta['qtde_evadidos'] > 0]
# Armazena o dataframe no banco de dados
df_consulta.to_sql('qry_qtde_alunos_evadidos_periodo', conexao_bd, index_label='id')
df_consulta

,periodo_letivo,nivel_ensino,modalidade_curso,curso,campus,qtde_evadidos
0,20091,Graduação,Bacharelado,BACHARELADO EM CIÊNCIA DA COMPUTAÇÃO,CAMPUS MARACANAÚ,10
1,20091,Graduação,Bacharelado,BACHARELADO EM ENGENHARIA AMBIENTAL,CAMPUS JUAZEIRO DO NORTE,1
2,20091,Graduação,Bacharelado,BACHARELADO EM ENGENHARIA AMBIENTAL E SANITÁRIA,CAMPUS MARACANAÚ,7
...,...,...,...,...,...,...
4884,20212,Técnico,Subsequente,TÉCNICO EM EDIFICAÇÕES,CAMPUS FORTALEZA,1
4885,20212,Técnico,Subsequente,TÉCNICO EM EDIFICAÇÕES,CAMPUS ITAPIPOCA,1
4887,20212,Técnico,Subsequente,TÉCNICO EM GEOPROCESSAMENTO,CAMPUS JUAZEIRO DO NORTE,1


In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['campus', 'cidade_campus', 'modalidade_curso', 'qtde_evadidos', 'qtde_matriculados'], \
                               as_index=False).agg({'qtde_evadidos': 'sum'})
# Retira os resultados zerados
df_consulta = df_consulta[df_consulta['qtde_evadidos'] > 0]
# Armazena o dataframe no banco de dados
df_consulta.to_sql('dat_qtde_matric_campus', conexao_bd, index_label='id')
df_consulta

TypeError: ignored

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['curso', 'modalidade_curso', 'qtde_matriculados'], \
                               as_index=False).agg({'qtde_matriculados': 'sum'})
# Retira os resultados zerados
df_consulta = df_consulta[df_consulta['qtde_matriculados'] > 0]
# Armazena o dataframe no banco de dados
df_consulta.to_sql('data_cursos', conexao_bd, index_label='id')
df_consulta

ValueError: ignored

### **6.7 Quantidade de Alunos Evadidos no Período Letivo por Campus**

In [ ]:
# Realiza uma consulta no dataset
df_consulta = df_dados.groupby(by=['campus', 'periodo_letivo'], \
                               as_index=False).agg({'qtde_matriculas_evadidas': pd.Series.sum})
# Renomeia a coluna da contagem
df_consulta.rename(columns={'qtde_matriculas_evadidas': 'qtde_evadidos'}, inplace=True)
# Armazena o dataframe no banco de dados
#df_consulta[df_consulta['qtde_evadidos']>0].to_sql('qry_qtde_alunos_evadidos_periodo', conexao_bd, index_label='id')
df_consulta

SpecificationError: Column(s) ['qtde_matriculas_evadidas'] do not exist